In [5]:
import numpy as np
import pandas as pd

# Zernike 다항식 계산 함수
def compute_zernike_matrix(r, theta, max_order=5):
    """
    Zernike 다항식 기저 생성
    :param r: 방사 좌표계의 반지름 값 (0 ≤ r ≤ 1로 스케일링 필요)
    :param theta: 방사 좌표계의 각도 값 (-π ≤ θ ≤ π)
    :param max_order: 최대 차수
    :return: Zernike 기저 행렬
    """
    Z = [np.ones_like(r)]  # Z0,0: 상수항
    for n in range(1, max_order + 1):
        for m in range(-n, n + 1, 2):
            if m >= 0:
                Z.append(r**n * np.cos(m * theta))  # 짝수 m: cos(mθ)
            else:
                Z.append(r**n * np.sin(-m * theta))  # 홀수 m: sin(-mθ)
    return np.array(Z).T  # 각 Zernike 다항식이 열로 구성된 행렬

# Zernike 회귀분석 함수
def zernike_regression(df_rawdata, max_order=5):
    grouped = df_rawdata.groupby('UNIQUE_ID')
    zernike_results = []
    
    for unique_id, group in grouped:
        # 좌표 변환
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        field_x = group['coordinate_X']
        field_y = group['coordinate_Y']

        
        wf_x = die_x * step_pitch_x + map_shift_x + field_x
        wf_y = die_y * step_pitch_y + map_shift_y + field_y
        r = np.sqrt(wf_x**2 + wf_y**2) / 1e6  # 거리 스케일링
        theta = np.arctan2(wf_y, wf_x)
        
        # Zernike 기저 행렬 생성
        Z = compute_zernike_matrix(r, theta, max_order=max_order)
        Z_list = pd.DataFrame(Z)
        print(Z_list)

        # 종속변수
        Y_dx = group['X_reg']
        Y_dy = group['Y_reg']

        # 최소자승법으로 계수 계산
        coeff_dx = np.linalg.lstsq(Z, Y_dx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(Z, Y_dy, rcond=None)[0]

        # 결과 저장
        result = {'UNIQUE_ID': unique_id}
        for i, coeff in enumerate(coeff_dx):
            result[f'Z{i+1}_dx'] = coeff
        for i, coeff in enumerate(coeff_dy):
            result[f'Z{i+1}_dy'] = coeff
        zernike_results.append(result)
    
    return pd.DataFrame(zernike_results)

# 예측값 및 잔차 계산 함수
def zernike_fitting_residual(df_rawdata, df_coeff, max_order=5):
    grouped = df_rawdata.groupby('UNIQUE_ID')
    predictions_list = []
    
    for unique_id, group in grouped:
        # 좌표 변환
        die_x = group['DieX']
        die_y = group['DieY']
        step_pitch_x = group['STEP_PITCH_X']
        step_pitch_y = group['STEP_PITCH_Y']
        map_shift_x = group['MAP_SHIFT_X']
        map_shift_y = group['MAP_SHIFT_Y']
        field_x = group['coordinate_X']
        field_y = group['coordinate_Y']

        
        wf_x = die_x * step_pitch_x + map_shift_x + field_x
        wf_y = die_y * step_pitch_y + map_shift_y + field_y
        r = np.sqrt(wf_x**2 + wf_y**2) / 1e6  # 거리 스케일링
        theta = np.arctan2(wf_y, wf_x)
        
                
        # Zernike 기저 행렬 생성
        Z = compute_zernike_matrix(r, theta, max_order=max_order)

        # 계수 추출
        coeff_row = df_coeff[df_coeff['UNIQUE_ID'] == unique_id].iloc[0]
        coeff_dx = coeff_row[[col for col in coeff_row.index if col.startswith('Z') and col.endswith('_dx')]].values
        coeff_dy = coeff_row[[col for col in coeff_row.index if col.startswith('Z') and col.endswith('_dy')]].values

        # 예측값 계산
        pred_x = Z @ coeff_dx
        pred_y = Z @ coeff_dy

        # 잔차 계산
        residual_x = group['X_reg'] - pred_x
        residual_y = group['Y_reg'] - pred_y

        # 결과 저장
        predictions_list.append(pd.DataFrame({
            'UNIQUE_ID': unique_id,
            'pred_x': pred_x,
            'pred_y': pred_y,
            'residual_x': residual_x,
            'residual_y': residual_y,
        }))
    
    return pd.concat(predictions_list, ignore_index=True)

# 데이터 불러오기
df_rawdata = pd.read_csv("RawData-1.csv")

# Zernike 회귀분석 수행
df_coeff = zernike_regression(df_rawdata, max_order=5)
df_coeff.to_csv('Zernike_Coefficients.csv', index=False)

# 예측값 및 잔차 계산
df_predictions = zernike_fitting_residual(df_rawdata, df_coeff, max_order=5)

# 원본 데이터와 결합
df_rawdata = pd.concat([df_rawdata, df_predictions], axis=1)
df_rawdata.to_csv('Z_FIT.csv', index=False)



       0         1         2         3         4         5         6   \
0     1.0 -0.002314 -0.132688  0.000614  0.017611  0.017601 -0.000122   
1     1.0  0.030626 -0.132688 -0.008127  0.018544  0.016668  0.001589   
2     1.0 -0.068194 -0.110104  0.015017  0.016773  0.007473 -0.002163   
3     1.0 -0.002314 -0.110104  0.000509  0.012128  0.012118 -0.000084   
4     1.0  0.030626 -0.110104 -0.006744  0.013061  0.011185  0.001085   
...   ...       ...       ...       ...       ...       ...       ...   
1889  1.0  0.016470  0.098724  0.003252  0.010018  0.009475  0.000477   
1890  1.0  0.049410  0.098724  0.009756  0.012188  0.007305  0.001324   
1891  1.0 -0.049410  0.121307 -0.011988  0.017157  0.012274 -0.002061   
1892  1.0 -0.016470  0.121307 -0.003996  0.014987  0.014444 -0.000723   
1893  1.0  0.016470  0.121307  0.003996  0.014987  0.014444  0.000723   

            7         8         9   ...        11        12        13  \
0    -0.000041 -0.002337 -0.002334  ...  0.000011 